In [6]:
#1 .env
import numpy as np
import pandas as pd
import fuzzywuzzy
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [7]:
#2. 
df_boxrec = pd.read_csv('data/boxrec_merge.csv')  # BoxRec data를 불러옴
df_boxrec.head()

,name,bouts,KOs
0,SAUL ALVAREZ,62,62.90
1,DMITRII BIVOL,20,55.00
2,TERENCE CRAWFORD,38,76.32
3,GENNADIY GOLOVKIN,45,82.22
4,OLEKSANDR USYK,20,65.00


In [8]:
#3.
df_boxstats = pd.read_csv('data/boxstats_merge.csv')  # BoxStats data를 불러옴
df_boxstats.head()

,name,bouts,KO %
0,ROAMER ALEXIS ANGULO,27,81.5
1,ALFREDO ANGULO,30,63.3
2,EFETOBOR APOCHI,3,100.0
3,FRED APOSTOLI,72,43.1
4,STEVE AQUILINA,22,27.3


In [9]:
#4. boxstats, boxrec의 이름들을 리스트로 만들어서 비교
boxstats_name = df_boxstats.name.tolist() 
boxrec_name = df_boxrec.name.tolist()
print(boxstats_name[:5], boxrec_name[:5])

['ROAMER ALEXIS ANGULO', 'ALFREDO ANGULO', 'EFETOBOR APOCHI', 'FRED APOSTOLI', 'STEVE AQUILINA'] ['SAUL ALVAREZ', 'DMITRII BIVOL', 'TERENCE CRAWFORD', 'GENNADIY GOLOVKIN', 'OLEKSANDR USYK']


In [10]:
#5. 서로 달라서 비교가 안되는 이름들을 리스트로 뽑아내기 (boxstats -> boxrec 비교)
li_nonmergable = []
for name in boxstats_name:
  if name not in boxrec_name:
    li_nonmergable.append(name)
    
print(len(li_nonmergable))

1469


In [11]:
#fuzzywuzzy의 ratio (사실상 levenshtein 유사도)를 이용해서 비교
fuzz_dict = {}
for name in li_nonmergable:
  for n in boxrec_name:
    ratio = fuzz.ratio(name, n)
    if ratio > 90:  # 점수가 90% 이상이면 뽑아옴
      if name in fuzz_dict.keys():
        fuzz_dict[name].append((n, ratio))
      else:
        fuzz_dict[name] = [(n, ratio)]

In [12]:
print(fuzz_dict)


{'RAEESE ALEEM': [("RA'EESE ALEEM", 96)], 'JERMALL CHARLO': [('JERMELL CHARLO', 93)], 'TED CHEESMAN': [('TED CHEESEMAN', 96)], 'DERECK CHISORA': [('DEREK CHISORA', 96)], 'KAREN CHUKHADZHYAN': [('KAREN CHUKHADZHIAN', 94)], 'JEZREEL CORRALES': [('JEZZREL CORRALES', 94)], 'CARLOS CRUZAT': [('CARLOS CRUZ', 92)], 'YUNIER DORTICOS': [('YUNIEL DORTICOS', 93)], 'ROMEO DUNO': [('ROMERO DUNO', 95)], 'AHMED ELBIALE': [('AHMED ELBIALI', 92)], 'RUSLAN FAYFERR': [('RUSLAN FAYFER', 96)], 'GABRIEL FLORES JR.': [('GABRIEL FLORES JR', 97)], "O'SHANIQUE FOSTER": [("O'SHAQUIE FOSTER", 91)], 'RAYMART GABALLO': [('REYMART GABALLO', 93)], 'GENNADY GOLOVKIN': [('GENNADIY GOLOVKIN', 97)], 'NICK HANNING': [('NICK HANNIG', 96)], 'LADUMO LAMATI': [('LUDUMO LAMATI', 92)], 'FRANCISCO MATEOS': [('FRANCISCO MARTOS', 94)], 'EMMANUEL NAVARRETE': [('EMANUEL NAVARRETE', 97)], 'ALEJANDRO SANTIAGO': [('ALEXANDRO SANTIAGO', 94)], 'GIOVANNI SANTILLAN': [('GIOVANI SANTILLAN', 97)], 'JAMES SMITH': [('JAMIE SMITH', 91)], 'JOSE 

#알고리즘 완료! 이를 활용해서 가장 높은 Levenshtein Distance ratio(fuzz ratio)를 갖는 이름을 찾아서 merge해주면 됨
#다만 위 경우는 boxrec 선수 갯수가 모자라서, 비슷한 이름의 다른 선수와 매칭되는 경우가 존재. 이를 해결하기 위해 boxrec에서 크롤링해오는 선수 갯수를 늘려야함